In [ ]:
#!/usr/bin/python3
# Description: Order Pizza Online

import random
import cgitb; cgitb.enable()
from datetime import date
import time
import cgi
import cgitb;cgitb.enable
import MySQLdb as db

######################################################################################################
def connect_to_database():
    '''Create a connection object to connect to MySQL database.
    Return the connection and cursor objects.
    '''
    conn = db.connect(host="localhost",
                  user="phuonghb", 
                  passwd="3704",
                  db="cs108_phuonghb_project")
   
    cursor = conn.cursor()
    return conn, cursor

######################################################################################################
def print_form_data(form):
    '''Display the form data for debugging purposes.
    '''
    # print out the form data for debugging purposes
    print('Form data:<br>')
    print('<table>')
    keys = list(form.keys())
    keys.sort()
    # go through all keys:
    for k in keys:
    	print('''<tr>
                    <td>%s</td>
                    <td>%s</td>
                </tr>''' % (k, form[k]))
    print('</table><hr>')

######################################################################################################

def print_headers():
    """Print the HTTP headers."""
    #print out HTTP Content-type header
    print("""Content-Type: text/html""")
    print()

######################################################################################################
def navigation_bar():
   """Print out the horizonal top navigation bar"""
   
   #designing navigation bar with color and font
   #then connect each button with link to pages
   
   print("""<!DOCTYPE html>
        <html>
        <head>
        
        <center><meta name="viewport" content="width=device-width, initial-scale=1">
        
        <style>
        
        body {margin: 0 0 0 0;
            font-family: Arial, Helvetica, sans-serif;}

        .topnav {overflow: hidden;
                background-color:#648560;}

        .topnav a {float: left;
                  position: relative;
                  color:white ;
                  text-align: center;
                  padding: 10px 40px;
                  text-decoration: none;
                  font-size: 18px;}

        .topnav a:hover {background-color: #648560;
                  color:#e7b04b;}
        </style>
        
        </head>
        
        <body>

        <div class="topnav">
          <a class="active" href="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py">Home</a>
          <a href="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py?about=True">About</a>
          <a href="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py?contact=True">Contact</a>
          <a class="active" href="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py?menu=Menu">Create Order</a>
          <a class="active" href="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py?order=True">View Your Order</a>


        </div>
        </body>
        </center>
        </html>
        """)
######################################################################################################
def print_top_of_page(title):
    """Print the title for webpage"""
    #print the title of the webpage
    print("""
        <html>
        <head>
        <title>%s</title>
      
        </head>
        <body>

		""" %title)
	    

######################################################################################################
def print_bottom_of_page():
    """Print the bottom of the page"""
    #print time stamps which indicates that the page is updated whenever the page is reloaded
    
    print('''<br>

    This page was generated at %s.<br>
    Return to the <a href="finalproject.py">main page</a>.
    </body>
    </html>
    ''' % time.ctime())
    
######################################################################################################
def print_front_page():
    """Print front page"""
    #print the first page of the web
    print("""
        <style>
        body {
        background-image: url("front_page_img.jpg");
        background-size:     cover;                     
        background-repeat:   no-repeat;
        background-position: center center;
        color:white;}
        </style>
        
        <body>
        
        <br><br><br><br><br><br><br><br><br><br><br><br><br>
        
        <h1 style="font-family:Harlow Solid Italic"><center> Welcome to Felicity's Pizza Shop</center> </h1><br>
        
        <center style ="font-family:Segoe Script"><b> THE BEST TASTING FOOD STARTS WITH INGREDIENTS<br>
        THAT ARE NATURALLY BETTER FOR YOU<b></center>

        </body>
        <br><br><br><br><br><br><br><br><br><br><br>
        """)
	
#############################################################################################################
def new_customer(C_firstname, C_lastname,C_email, shipping_address):
    """Check whether the customer is new or already existed in the database"""
    
    # connect to database
    conn, cursor = connect_to_database()
    
    # build SQL
    sql = """
    SELECT C_firstname, C_lastname
    FROM Customers
    WHERE C_firstname = "%s" and C_lastname = "%s"  
    """%(C_firstname, C_lastname)

    # execute the query
    cursor.execute(sql)

    # get the data from the database:
    data = cursor.fetchall()
    
    #len(data)== 0 means that the customer has not existed yet in the database
    if len(data) == 0:
        sql="""INSERT INTO Customers(C_ID, C_firstname, C_lastname,C_email, shipping_address)
            VALUES (NULL, "%s", "%s", "%s", "%s")""" % (C_firstname, C_lastname,C_email, shipping_address)
        cursor.execute(sql)
        conn.commit()
        sql= """SELECT last_insert_ID()"""
        
        #execute query under if 
        cursor.execute(sql)
        
        # clean up under if
        conn.close()
        cursor.close()        
        return
    
    #lens(data) is not 0 means that the customer has existed in the database
    else:
        #clean up under else
        conn.close()
        cursor.close()
        return
################################################################################################################
def get_customer_data(C_firstname, C_lastname):
    """Check whether the customer is new or already existed in the database"""

    # connect to database
    conn, cursor = connect_to_database()

    # build SQL
    sql = """
    SELECT *
    FROM Customers
    WHERE C_firstname = "%s" and C_lastname = "%s"  
    """%(C_firstname, C_lastname)

    # execute the query
    cursor.execute(sql)
    data=cursor.fetchall()
    conn.close()
    cursor.close()
    return data[0]

######################################################################################################
def get_customer_ID(C_firstname, C_lastname):
    """Check whether the customer is new or already existed in the database"""
    #get customer ID corresponding to C_firstname, C_lastname
    data = get_customer_data(C_firstname,C_lastname)
    return data[0]

######################################################################################################
def new_order(C_ID, total_price):
    """Inserts a new profile into the website"""
    #randomly generate tracking_number
    tracking_number = random.randint(10000,99999)
    
    #return the order date using following function
    order_date = date.today().strftime("%Y-%m-%d")
    
    # connect to database
    conn, cursor = connect_to_database()
    
    # build SQL
    sql = '''
    INSERT INTO Orders (order_id, C_ID, order_date, tracking_number, total_price)
    VALUES (NULL, %s, %s, %s, %s)''' 

    parameter = (C_ID, order_date,tracking_number, total_price)
    
    # execute the query
    cursor.execute(sql,parameter)
    
    #clean up
    conn.commit()
    cursor.close()
    conn.close
    
    return tracking_number,order_date

######################################################################################################
def get_order_ID(C_ID,tracking_number,order_date ):
    """Check whether the customer is new or already existed in the database"""

    # connect to database
    conn, cursor = connect_to_database()

    # build SQL
    sql = """
    SELECT *
    FROM Orders
    WHERE C_ID= "%s" and tracking_number= "%s"  and order_date="%s"
    """%(C_ID, tracking_number, order_date)

    # execute the query
    cursor.execute(sql)

    # fetch
    data=cursor.fetchall()

    # clean up
    conn.close()
    cursor.close()
    
    return data[0][0]


######################################################################################################
def get_item_ID(item):
    """ Retrieve price of each item from the database."""
    # connect to database
    conn, cursor = connect_to_database()

    # build SQL
    sql = """
    SELECT Item_ID
    FROM Pizza
    WHERE Item=%s
    """

    # execute the query
    parameters = (item,)
    cursor.execute(sql, parameters)

    # get the data from the database:
    data = cursor.fetchall()

    # clean up
    cursor.close()
    conn.close()
    
    return data
######################################################################################################
def new_orderdetails(order_id, pizza):
    """Insert a new order details into the website"""
    # connect to database
    conn, cursor = connect_to_database()    
    for x in pizza:
        
    #check if item is list
        if type(x) == list:
                for i in x:
                       Item_ID = get_item_ID(i)
                       
                       #write SQL query
                       sql = '''
                            INSERT INTO Order_details (Order_details_id, order_id, Item_ID)
                            VALUES (NULL, %s, %s)'''
                       
                       parameter = (order_id, Item_ID)

                       #execute SQL
                       cursor.execute(sql, parameter)
        else:
                Item_ID= get_item_ID(x)
                
                #write SQL query
                sql = '''
                    INSERT INTO Order_details (Order_details_id, order_id, Item_ID)
                    VALUES (NULL, %s, %s)''' 

                parameter = (order_id, Item_ID)

                #execute SQL
                cursor.execute(sql, parameter)
    #fetch
    rc = cursor.rowcount

    #clean up
    conn.commit()
    cursor.close()
    conn.close
    
    return rc
    
######################################################################################################
def print_order():
	"""Print the order details"""
	#print order details form for the customer to fill
	#then print the information form for the customer to fill
	print("""<html>

	<head>
	
	<title> Pizza Love </title>
	
	</head>
        <style>
        body {
        background-image: url("createorder1.jpg");
	background-size: cover;                  
        color:white;}
        </style>


        <body>
        
	<h1 style="color:black"><center>Build your own pizza</center></h1>
	
	<form>
	
	<table width="100%">
	
	     <tr>
			<th width = "10%"> Choose your size</th>
			<td height=200 width="10%">
			<input type ="radio" name = "size" value = "S"  <label>Small</label><br>
			<input type = "radio" name = "size" value = "M" <label>Medium</label><br>
			<input type = "radio" name = "size" value = "L" <label>Large</label><br>
			</td>
		
			<th width = "10%"> Choose your Cheese</th>
			<td width="10%">
			<input type = "radio" name = "cheese" value = "Provolone" <label>Provolon</label><br>
			<input type = "radio" name = "cheese" value = "Cheddar" <label>Cheddar</label><br>
			<input type = "radio" name = "cheese" value = "Parmigiano" <label>Parmigiano Parmesan</label><br>
			</td>

			<th width = "10%"> Choose your Sauce</th>
			<td width="10%">
			<input type = "radio" name = "sauce" value = "Tomato" <label>Tomatoes</label><br>
			<input type = "radio" name = "sauce" value = "BBQ" <label>BBQ</label><br>
			<input type = "radio" name = "sauce" value = "Garlic Parmesan" <label>Garlic Parmesan</label><br>
			</td>
			
			<th width = "10%"> Choose your Meat</th>
			<td width="10%">
			<input type = "checkbox" name = "meat" value = "Ham" <label>Ham</label><br>
			<input type = "checkbox" name = "meat" value = "Beef" <label>Beef</label><br>
			<input type = "checkbox" name = "meat" value = "Salami" <label>Salami</label><br>
			<input type = "checkbox" name = "meat" value = "Pepperoni" <label>Pepperoni</label><br>
			<input type = "checkbox" name = "meat" value = "Italian Sausage" <label>Italian Sausage</label><br>
			</td>

			<th width = "10%"> Choose your Non-Meat</th>
			<td width="10%">
			<input type = "checkbox" name = "nonmeat" value = "Onions" <label>Onions</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Diced Tomatoes" <label>Diced Tomatoes</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Black Olives" <label>Black Olives</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Mushrooms" <label>Mushrooms</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Pineapple" <label>Pineapple</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Roasted Red Peppers" <label>Roasted Pepper</label><br>
			</td>
	   </tr>
		
			
	</table>
	
        <table>
        
        <h1 style="color:black">Sign in to your Pizza Profile</h1>
        
        <b>Please fill completely fill out the following:</b>
        
        <p>
        <form>
        <table>
            <tr>
                <td><b>First Name:</b></td>
                <td><input type='text' name='C_firstname' size='55'></td>
            </tr>
            <tr>
                <td><b>Last Name:</b></td>
                <td><input type='text' name='C_lastname' size='55'></td>
            </tr>
            <tr>
                <td><b>Shipping Address:</b></td>
                <td><input type='text' name='shipping_address' size='55'></td>
            </tr>
            <tr>
                <td><b>Email Address:</b></td>
                <td><input type='text' name='C_email' size='55'></td>
            </tr>
        </table>
        
            <tr>
                    <td></td>
			<center><input type='submit' name='item' value='Process Order'></center>
                    </td>
	    </tr></th> 
                     
            </form>
            <style> body{background-size:cover} </style>
            </body>
            """)
	
######################################################################################################
def total_price(list):
    """Return the total price of the list of
    pizza item that the customer chooses"""
    price = 0
    #run the loop through each item
    for x in list:
        price+=x
    return price

######################################################################################################
def item_price(item):
    """ Retrieve price of each item from the database."""
    # connect to database
    conn, cursor = connect_to_database()

    # build SQL
    sql = """
    SELECT Price
    FROM Pizza
    WHERE Item=%s
    """

    # execute the query
    parameters = (item,)
    cursor.execute(sql, parameters)

    # get the data from the database:
    data = cursor.fetchall()

    # clean up
    cursor.close()
    conn.close()
    
    return data

######################################################################################################	
def price_item_list(pizza):
    list1 = []
    
    #run the loop through each item if there is or if there isn't a list
    for i in pizza:
        if type(i) == list:
                for x in i:
                        list1.append(item_price(x)[0][0])
        else:
                list1.append(item_price(i)[0][0])
    return list1

######################################################################################################	
def print_selectedorder(C_firstname, C_lastname, shipping_address, C_email,cheese,sauce,size,meat,nonmeat,total_price, C_ID,order_id):
    """Display customer's information and customer's choice of pizza."""
    #display customer's choice of pizza and give options to edit/cancel/keep the order
    message="""
            <body background="confirm.jpg"; style = "color: white">
            
            <style="background-color:#e05038;"style = "color: white"><br>
            
            <h3>Your shipping information is below:</h3><br>
            <b>Your name:</b> %s %s <br>
            <b>Your shipping address:</b> %s<br>
            <b>Your email address:</b> %s<br><br><br>
            
            <h3>You selected:</h3><br>
            Size: %s<br>
            Cheese: %s<br>
            Sauce: %s<br>
            Meat: %s<br>
            Toppings: %s<br>
            
            We'll keep your cart ready for you!<br>
            
            Your total price is $%s<br><br><br><br>
            
            <form>
            
            <table border= 5; bgcolor="7e9f7a">
                <th><b>Do you want to change your order?</b></th>
                
                <td>
                <select name="option">
                  <option value="update">Yes, I want to update my order</option>
                  <option value="cancel">Yes, I want to cancel my order</option>
                  <option value="keep">No, I want to keep my order</option>
                </select>
                </td>
                
                <tr>
                    <td><input type='submit' name='process' value='Process Request'></td>
                    <input type='hidden' name='C_ID' value=%s>
                    <input type='hidden' name='order_id' value=%s>
                    <input type='hidden' name="size" value=%s>
                    <input type='hidden' name='cheese' value=%s>
                    <input type='hidden' name='sauce' value=%s>
                    <input type='hidden' name='meat' value=%s>
                    <input type='hidden' name='nonmeat' value=%s>

                </tr>
                
            </table>
            </form>
            </body> 
            """%(C_firstname, C_lastname, shipping_address, C_email, size, cheese, sauce, ", ".join(meat), ", ".join(nonmeat),total_price, C_ID,order_id,size,cheese.replace(" ","_"),sauce.replace(" ","_"),",".join(meat).replace(" ","_"),",".join(nonmeat).replace(" ","_"))
    #since we want to bring meat and nonmeat value to the next page, we need to turn them into string and assign them into hidden input. since they are a list, we have to join them with underscore and comma
    print(message)

######################################################################################################
def form_update_order(checkbox):
    """Update customer's order"""
    #redisplay the menu and print a form for the customers to update their orders
    
    message = '''
    <body background="confirm.jpg"; style = "color: white">
    
    <h2>Update Your Pizza</h2>

    Fill this form to update your pizza
    
        <form>
        
	<table cellspacing="0" cellpadding="0" width="100%"; bgcolor="#e6af4b">
	
                    <tr>
                        
                        <th width = "10%" bgcolor="#FBCB7B"> Choose your <br> size</th>
			<td height=200 width="8%" bgcolor="#FBCB7B">
			<input type ="radio" name = "size" value = "S"  %s><label>Small</label> <br>
			<input type = "radio" name = "size" value = "M" %s><label>Medium</label><br>
			<input type = "radio" name = "size" value = "L" %s><label>Large</label><br>
			</td>

                        <th width = "10%" bgcolor="#FD974F"> Choose your <br> Cheese</th>
			<td width="9%" bgcolor="#FD974F">
			<input type = "radio" name = "cheese" value = "Provolone" %s><label>Provolon</label><br>
			<input type = "radio" name = "cheese" value = "Cheddar" %s><label>Cheddar</label><br>
			<input type = "radio" name = "cheese" value = "Parmigiano" %s><label>Parmigiano Parmesan</label><br>
			</td>
		

			<th width ="10%"   bgcolor="#7F152E"> Choose your <br> Sauce</th>
			<td width="10%" bgcolor="#7F152E">
			<input type = "radio" name = "sauce" value = "Tomato" %s><label>Tomatoes</label><br>
			<input type = "radio" name = "sauce" value = "BBQ" %s><label>BBQ</label><br>
			<input type = "radio" name = "sauce" value = "Garlic Parmesan" %s><label>Garlic Parmesan</label><br>
			</td>
			
			<th width = "10%" bgcolor="#C60000"> Choose your <br> Meat</th>
			<td width="10%" bgcolor="#C60000">
			<input type = "checkbox" name = "meat" value = "Ham" %s><label>Ham</label><br>
			<input type = "checkbox" name = "meat" value = "Beef" %s><label>Beef</label><br>
			<input type = "checkbox" name = "meat" value = "Salami" %s><label>Salami</label><br>
			<input type = "checkbox" name = "meat" value = "Pepperoni" %s><label>Pepperoni</label><br>
			<input type = "checkbox" name = "meat" value = "Italian Sausage" %s><label>Italian Sausage</label><br>
			</td>

			<th width = "10%" bgcolor="#805A3B"> Choose your <br> Non-Meat</th>
			<td width="12%" bgcolor="#805A3B">
			<input type = "checkbox" name = "nonmeat" value = "Onions" %s><label>Onions</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Diced Tomatoes" %s><label>Diced Tomatoes</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Black Olives" %s><label>Black Olives</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Mushrooms" %s><label>Mushrooms</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Pineapple" %s><label>Pineapple</label><br>
			<input type = "checkbox" name = "nonmeat" value = "Roasted Red Peppers" %s><label>Roasted Red Pepper</label><br>
			</td>
                    </tr>
        </table>
        <br><br><br><br><br>
                        
        <tr>
        <td></td>
        <center><input type='submit' name='newitem' value='Update Order'></center>
        <input type='hidden' name='order_id' value=%s>
        </td>
        </tr></form><body>
           '''
    for i in range(len(checkbox)):
       message = message.replace("%s",checkbox[i],1)
    print(message)

###################################################################################################### 
def check_checked(list,order_id):
    """Keep the previous choice of order
    prefilled in the update page/form"""
    # 1. Connect to database, obtain cursor object

    conn, cursor = connect_to_database()
    checkbox = [' ' for i in range(20)]

    # 2. Write SQL query as a Python string
    # run a loop through a list of items in the pizza and select the item id
    for i in list:
        sql = """
            SELECT Item_ID
            FROM Pizza
            WHERE Item=%s
            """
        parameters = (i,)
    # 3. Execute the SQL query against the database cursor
        cursor.execute(sql, parameters)
        
    # 4. Fetch/process the result set
        data = cursor.fetchall()
        
    # 5. Clean up
        checkbox[int(data[0][0])-1] = "checked"
    checkbox.append(order_id)
    cursor.close()
    conn.close()
    
    return checkbox

######################################################################################################
def update_order(order_id,  pizza):
    """Update the existing order details corresponding to order_id and
    return a row count of how many records were affected
    """
    # Connect to database, obtain cursor object
    conn, cursor = connect_to_database()

    # Write SQL query as a Python string
    sql = """
    DELETE FROM Order_details
    WHERE order_id =%s
    """
    
    parameter= (order_id,)
    
    # Execute the SQL query against the database cursor
    cursor.execute(sql,parameter)

    # Clean up
    cursor.close()
    conn.commit()
    new_orderdetails(order_id, pizza)
    rowcount = cursor.rowcount
    
    conn.close()
    
    return rowcount

###################################################################################################### 
def delete_order(order_id):
    """Delete the existing order details corresponding to order_id
    by deleting existed order and insert new order with the same order id but with new pizza items.
    """
    # Connect to database, obtain cursor object
    conn, cursor = connect_to_database()

    # Write SQL query as a Python string
    sql1= """
    DELETE FROM Order_details
    WHERE order_id =%s
    """
    
    parameter= (order_id,)

    # Execute the SQL query against the database cursor
    cursor.execute(sql1,parameter)

    # Write SQL query as a Python string
    sql2="""
    DELETE FROM Orders
    WHERE order_id=%s
    """ 
    parameter= (order_id,)

    # Execute the SQL query against the database cursor
    cursor.execute(sql2,parameter)


    # Fetch
    rowcount = cursor.rowcount

    # Clean up
    cursor.close()
    conn.commit()
    conn.close()
    
    return rowcount

###################################################################################################### 
def confirmation_page(data):
    """Display a confirmation message."""
    #print confirmation page with order id number
    print("""
        <body background="thankyou.jpg"; style = "color: black">
        
        <h2>Confirmation Page</h2>
        
        <p>
        
        <body style="background-color:#ffecec";>
        
        <h2>Thank you for choosing our pizza!</h1>
        
        <b><h2>Your Order ID is: %s

        </body></h2><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
        """%(order_id))
    	
	
######################################################################################################	
def error_form1():
    """Print error message when the checkboxes left blank
    """
    #error message when customer leaves pizza checkboxes blank
    message = """
    <h2> You must not leave any checkboxes blank</h2>
    """
    print(message)
    return

######################################################################################################	
def error_form2():
    """Print error message when the customer's information left blank
    """
    #error message when customer leaves customer's information fields blank
    message = """
            <h2> You must not leave any fields blank</h2>
            """
    print(message)
    return

######################################################################################################
def error_button1():
    """Print the error button"""
    #error button that take the customer back to create order page to reorder.
    message="""

            <body style="background-color:#eaba62;">
            <form>
            
            <h1 style="color:black">Error!!!</h1><br>
            
            <input type='submit' name='menu' value='Back to Menu'>
            
            </form>
            </body>
            """
    print(message)

######################################################################################################
def error_button2():
    """Print the error button"""
    #error button that take the customer back to update order page
    message="""

            <body style="background-color:#eaba62;">
            
            <h1 style="color:black">Error!!!</h1><br>
            
            <button onclick="goBack()">Back to Update</button>
            <script>
            function goBack(){window.history.back();}
            </script>
            
            </body>
            
            """
    print(message)

######################################################################################################
def ab():
    """Print about us page"""
    #display a page shows the restaurant information with format and colors
    message="""
        <form>
        
   	<style> body  {
  	background-image: url("about.jpg");
  	background-size: cover;                   
   	background-repeat:   no-repeat;
   	background-position: center center;}
   	.blend {color: white;
 	 mix-blend-mode: difference;}
 	</style>
 	
        <body>
        
        <div class="blend">
        
        <h1><center style="font-family:Yellowtail"> About Felicity's Pizza Shop</center> </h1><br>
	<center  style="font-family:Segoe Script"> Deep-seated in a Pizza Restaurant is a Dream to Create </center><br>
	<center  style="font-family:Segoe Script"> the Place to Spread Smiles Around the World</center><br>
	
        Serving delicious food for customers to enjoy is a given for a restaurant.<br>
        We strive to be more than just a restaurant.<br>
        We want every person who steps into our world to be more excited, to feel happier.<br>
        
        <p>
        
        <h1><center> Our Missions</center> </h1><br>
        
        <p style= "color:white" style="background-color:#e05038"> Great food comes from great ingredients.<br>
        
        That's why we only use organic ingredients.<br>
        We never use ingredients that contain MSG, high fructose corn syrup,<br>
        artificial colors and flavorings, hormones or antibiotics.<br>
        We freeze our food so that we never have to use preservatives or shelf stabilizers. <br>
        We avoid gluten so that large number of people with dietary restrictions can enjoy our the food we cook. <br>
        We hope when it comes to Felicty's Pizza Shop, you will know that you always have unadulterated goodness that only nature can provide.

        </p></span>
        
        <br><br><br><br><br>
        
	</div>
	
        </body></form>"""
    print(message)

######################################################################################################
def contact():
    """Print the contact page
    """
    
    #display the page that shows the restaurant contact information with colors and format

    message="""
    <body center=True background="contact.jpg"; style = "color: #334431;">
     <p>
    <h1 style= color:black>Stop by and say "Hi"<br>
    Or call us.</h1><br><br>
    <p>
    <b>Thank you for choosing Felicity's Pizza Shop!<br><br>
    We actively use feedback to constantly improve our delivery<br>
    and provide you with the best possible service.<b><br><br><br>
    
    <table border= 5; bgcolor="#e7b04b">
    <td>
    	If this is an urgent matter, please contact Customer Support:
    	<br><br>
    	Monday-Friday 9:30 AM- 10:00PM EST
   	<br><br>
    	<b>U.S.- 877-235-1602</b>
    </td>
    </table>
    """
    print(message)


################################################################################
def find_order():
    """Find customer's order by retrieve the customer's order id
    """
    
   #display a page that allows customer to see their orders by entering their order id number
    message="""
        <center><body background="createorder.jpg"; style = "color:white;">
        <h1 font size="5" color="white">View Your Pizza Order</h1>
        <font size="4" color="white"><b> Please enter your Order ID:</font><b>
        <p>
        <form>
        <table>
            <tr>
                <td><b>Your Order ID:<b></td>
                <td><input type='text' name='order_id' size='20'></td>
            </tr>
            <tr>
                <td></td>
	        <td><input type='submit' name='view' value='View'></td>
            </tr>
        </table><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
                     
            </form></center> 
            <style> body{background-size:cover} </style>
            """
    print(message)


################################################################################

def get_your_order(order_id):
    """Display customer's order details by retrieve the customer's order id
    """
    # connect to database
    conn, cursor = connect_to_database()
   
   # build SQL
    sql = """
    SELECT Pizza.Item, Pizza.Price, Orders.total_price
    FROM Order_details
    INNER JOIN Pizza ON  Order_details.Item_ID = Pizza.Item_ID
    INNER JOIN Orders ON Orders.order_id= Order_details.order_id
    WHERE Orders.order_id = "%s"  
    """%(order_id)
   
   # execute the query
    cursor.execute(sql)
   
   # get the data from the database:
    data = cursor.fetchall()
   
   # clean up
    conn.close()
    cursor.close()

    return data

################################################################################
def no_data():
    """Print error message page when the order id that
    the customer filled in view order page does not exist
    """
    
    #display an error page when the order id that the customer enter is not correct
    #print the button that take the customer back to view order page
    
    message="""
    <center><body style="background-color:#eaba62;">
    
     <p>
     <p>
     
    <b><font size="7" color="white">Retrieve order details failed.<br>
    <p>Your Order ID does not exist !!!<b></font><br>
    
    <table>
    <tr>
    <td></td>
    
    <form action="http://cs-webapps.bu.edu/cgi-bin/cs108/phuonghb/finalproject.py?order=True" method="get">
    <input type='submit' name='order' value='Try Again'>
    </form>
    
    </tr>
    </table></center>"""
    
    print(message)

    
################################################################################

def display_order(data):
    a= data[0]
    
    #print the page format
    print("""
    <style> body  {
  	background-image: url("createorder.jpg");
  	background-size: cover;                   
   	background-repeat:   no-repeat;
   	background-position: center center;}
   	.blend {
	 color: white;
 	 mix-blend-mode: difference;
	}
    </style>
    
    <p>
    
    <h1 style= color:white><center>Your Order Information Is Below:</center></h1><br><br>
    <center><table border= 3 width= 50%; bgcolor="white">
    
    <tr>
        <th> Your Pizza Toppings</th>
        <th> Toppings' Price</th>
    </tr>""")
    
    #print item name and item's price according to the customer's choice of pizza/order
    for record in data:
        # each iteration of this loop creates on record of output:
        (item, price, total_price) = record
        print("""
      <tr>
        <td>%s</td>
        <td>%s</td>
      </tr>""" % (item, price))

    #print the total price of the customer's pizza    
    print("""
    <tr>
        <td><b>Your Total Price<b></td>
        <td><b>%s<b></td>
    <tr></table>"""%(total_price))

######################################################################################################
def update_price(total_price,order_id):
    """Update the existing price corresponding to order_id and
    return a row count of how many records were affected
    """
    # 1. Connect to db, obtain cursor object
    conn, cursor = connect_to_database()


    # 2. Write SQL query in Python string format
    sql = '''
    UPDATE Orders
    SET total_price=%s
    WHERE order_id =%s
    '''
    
    parameter= (total_price, order_id)

    # 3. Execute the SQL query against the database cursor
    cursor.execute(sql,parameter)

    # 4. Fetch
    rowcount = cursor.rowcount

    #5. Clean up
    cursor.close()
    conn.commit()
    conn.close()
    
    return rowcount
   
######################################################################################################

if __name__ == "__main__":


    #top of the page stuff
    print_headers()
    navigation_bar()

    # obtain the HTTP form data into a python variable
    form = cgi.FieldStorage() #dictionary-like object

    # debugging information about form data
    #print_form_data(form) 
    
    #customer wants to view menu 
    if "menu" in form:
        print_order()
        
    #user wants to create order
    elif "item" in form:
        
        #test if there's an error then display the error message
        try:
            
            #test if the customer does not fill all the required information fields
            try:                
                #read HTTP form data element into python variable
                C_firstname= form["C_firstname"].value
                C_lastname= form["C_lastname"].value
                shipping_address= form["shipping_address"].value
                C_email= form["C_email"].value
                
                #insert customer data into customer table
                new_customer(C_firstname, C_lastname, C_email, shipping_address)
                C_ID = get_customer_ID(C_firstname, C_lastname)
                
            #display error page if the user does not fill all the required information fields
            except:
                error_form2()

            #test if the user does not fill all the required pizza checkboxes
            try:
                #read HTTP form data element into python variable
                cheese= form["cheese"].value
                sauce= form["sauce"].value
                size= form["size"].value
                #meat and non meat are checkbox items
                if "meat" in form:
                    if type(form["meat"]) == list:
                        meat = [i.value for i in form["meat"]]
                    else:
                        meat = [form["meat"].value]
                if "nonmeat" in form:
                    if type(form["nonmeat"]) == list:
                        nonmeat = [i.value for i in form["nonmeat"]]
                    else:
                        nonmeat = [form["nonmeat"].value]
                #find total price of the all selected item
                pizza = [cheese,sauce,size,meat,nonmeat]
                pizzalist= price_item_list(pizza)
                total_price= total_price(pizzalist)

            #display error page if the user does not fill all the required pizza checkboxes
            except:
                error_form1()

           
                
            #fill in orders table
            tracking_number, order_date = new_order(C_ID, total_price)

            #fill in order details table
            order_id= get_order_ID(C_ID, tracking_number, order_date )
            new_orderdetails(order_id, pizza)

            #print selected order confirmation
            print_selectedorder(C_firstname, C_lastname, shipping_address, C_email,cheese,sauce,size,meat,nonmeat,total_price, C_ID,order_id)
                
        #display error button for customer to come back to menu and reorder                                          
        except:
            error_button1()
            

    #customer wants to process and request new action to the order(keep/ update/ cancel order)
    elif 'process' in form:
        
        #customer wants to update order
        if form["option"].value== "update":

            #test if the user does not fill all the required pizza checkboxes
            try:
                #read HTTP form data element into python variable
                cheese= form["cheese"].value.replace("_"," ")
                sauce= form["sauce"].value.replace("_"," ")
                size= form["size"].value
                meat = form["meat"].value.replace("_"," ").split(",")
                nonmeat = form["nonmeat"].value.replace("_"," ").split(",")
                
                pizza = []
                pizza.append(cheese)
                pizza.append(sauce)
                pizza.append(size)
                for i in meat:
                   pizza.append(i)
                for i in nonmeat:
                   pizza.append(i)
                order_id = form['order_id'].value
                checkbox1 = check_checked(pizza,order_id)
                form_update_order(checkbox1)
                
            #display error page if the user does not fill all the required pizza checkboxes
            except:
                error_form1()
                
        #customer wants to cancel/delete order
        elif form["option"].value == "cancel":
            order_id = form['order_id'].value
            delete_order(order_id)
            print_front_page()
            print_top_of_page("order online")

        #customer wants to keep order
        else:
            order_id= form["order_id"].value
            confirmation_page(order_id)
            
    #customer wants to submit updated order
    elif 'newitem' in form:
        #test if the user does not fill all the required pizza checkboxes
        try:
            #read HTTP form data element into python variable
            cheese= form["cheese"].value
            sauce= form["sauce"].value
            size= form["size"].value
            if "meat" in form:
                if type(form["meat"]) == list:
                    meat = [i.value for i in form["meat"]]
                else:
                    meat = [form["meat"].value]
            if "nonmeat" in form:
                if type(form["nonmeat"]) == list:
                    nonmeat = [i.value for i in form["nonmeat"]]
                else:
                    nonmeat = [form["nonmeat"].value]
            
            pizza = [cheese,sauce,size,meat,nonmeat]
            pizzalist= price_item_list(pizza)
            total_price= total_price(pizzalist)
            order_id= form['order_id'].value
            update_order(order_id, pizza)
            update_price(total_price,order_id)
            confirmation_page(order_id)
            
        #display error page if the user does not fill all the required pizza checkboxes
        except:
            error_form1()
            error_button2()
            
    #customer wants to view about page
    elif 'about' in form:
        ab()
        
    #customer wants to view contact page
    elif 'contact' in form:
        contact()
        
   

    #customer wants to find their order by entering their order id
    elif 'view' in form:
        try:
            order_id= form["order_id"].value
            data= get_your_order(order_id)
            display_order(data)
        except:
            no_data()
    #display error page if customer entered the wrong/non-existed order id
    elif 'order' in form:
        find_order()


    #else the first page    
    else:
        print_front_page()
        print_top_of_page("order online")

    print_bottom_of_page()
   
    
	



